In [2]:
import pandas as pd
import s3fs
import boto3
from io import StringIO # python3; python2: BytesIO 
from boto3.s3.transfer import TransferConfig
import torch
from transformers import *
import numpy as np
import re
from langdetect import detect
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer

In [18]:
pd.set_option('display.max_colwidth', -1)

# The goal: Get text corresponding to train set and validation set and Language corresponding to it.

# Output should look like: Cleaned text and Language corresponding to it.

#### Divide timesamp and Language into train and validation set

In [6]:
def tokenize_text(row, index):
    if index % 100000 == 0:
        print(index)
    row1 = row.replace('[CLS]','')
    row2 = row1.replace('[SEP]', '')
    result = re.sub(r"http\S+", "", row2)
    tokenizer = RegexpTokenizer(r'\w+')
    token_list = tokenizer.tokenize(result)
    feature =  ' '.join([w.lower() for w in token_list])
    return feature

In [ ]:
def language_detect(str1, str2):
    return str1,detect(str2)

In [ ]:
all_features = ["Language", "tweet_timestamp", "engaging_user_id", "reply_timestamp"]

train = pd.read_csv("s3://recsys-challenge-2020/training.tsv", encoding="utf-8",
                    names = all_features, usecols= [7, 8, 14, 20], sep="\x01")
train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()
n_head = 90
n_tail = 10
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100))).reset_index()
train_set_text = pd.read_csv('s3://recsys-challenge-2020/train_set_text.csv')
val_set_text = pd.read_csv('s3://recsys-challenge-2020/val_set_text.csv')

In [ ]:
train_lda_input = pd.concat([train_set, train_set_text], axis=1)

In [ ]:
val_lda_input = pd.concat([val_set, val_set_text], axis=1)

In [ ]:
train_lda_input.to_csv('s3://recsys-challenge-2020/train_lda_input.csv', index = False)

In [ ]:
val_lda_input.to_csv('s3://recsys-challenge-2020/val_lda_input.csv', index = False)

In [3]:
train_lda_input = pd.read_csv('s3://recsys-challenge-2020/train_lda_input.csv')

In [4]:
cleaned_text_train = pd.DataFrame()
#cleaned_text_val = pd.DataFrame()

In [ ]:
cleaned_text_train['tweet_text'] = train_lda_input.apply(lambda x: tokenize_text(x.user_text, x.name), axis = 1)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000

In [10]:
cleaned_text_train.to_csv('s3://recsys-challenge-2020/cleaned_text_train.csv', index = False)

In [11]:
cleaned_text_train.head()

,tweet_text
0,envivo buenas noches comienza una nueva edició...
1,celebrate lunar new year with the new tigeress...
2,the media tend not to feature abortion survivo...
3,porto alegre tem novidade na agenda unk 14 de ...
4,karnataka boy who guided ambulance during floo...


In [9]:
len(cleaned_text_train)

133267714

In [13]:
cleaned_text_val = pd.DataFrame()

In [15]:
val_lda_input = pd.read_csv('s3://recsys-challenge-2020/val_lda_input.csv')

In [16]:
cleaned_text_val['tweet_text'] = val_lda_input.apply(lambda x: tokenize_text(x.user_text, x.name), axis = 1)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000

In [ ]:
group_train= train_lda_input.groupby('Language').first().reset_index()

In [19]:
cleaned_text_val.head()

,tweet_text
0,sm 사옥빌딩 앞에 있는 전광판차 사진들입니다 전광판차는 사옥 앞에서 오늘 아침 9시부터 오후 4시까지 있을 예정입니다 weareoneexo exo 엑소 t co cs70eh6ikt
1,방송 막하지 말라던 최민호는 본인이 막히기 시작하는데 t co wqz58zzz5l
2,rt reportertelly let s just check it out according to you who is the most deserving contestant among these two asimriaz sidharthsh unk
3,abhirupkumardu1 जय म
4,rt lovablebh _ 0506 백현이 오늘 심각하게 unk 던지고 가만히 서서 빵싯 웃는 거 unk 뛰어 내려갈 때는 또 얼마나 unk t co j9e1xykief


In [20]:
cleaned_text_val.to_csv('s3://recsys-challenge-2020/cleaned_text_val.csv', index = False)

# Work with test data now to get the clean text of test data

In [21]:
all_features = ["text_ tokens"]

test = pd.read_csv("s3://recsys-challenge-2020/val.tsv", encoding="utf-8",
                    names = all_features, usecols= [0], sep="\x01")

In [23]:
test_text = pd.DataFrame()

In [24]:
def calculate_text(row):
    tweet_tokens = tokenizer.decode(list(map(int, row.split('\t'))))
    return tweet_tokens

In [26]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

In [27]:
test_text['user_text'] = test['text_ tokens'].apply(lambda x: calculate_text(x))

In [28]:
test_text.to_csv('s3://recsys-challenge-2020/test_set_text.csv', index = False)

In [29]:
cleaned_text_test = pd.DataFrame()

In [30]:
cleaned_text_test['tweet_text'] = test_text.apply(lambda x: tokenize_text(x.user_text, x.name), axis = 1)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000

In [31]:
cleaned_text_test.to_csv('s3://recsys-challenge-2020/cleaned_text_test.csv', index = False)

In [32]:
cleaned_text_test.head()

,tweet_text
0,funky techno witch t co ydfhit7ncn
1,空 港 で 財 布 置 き 忘 れたら 偶 然 隣 座 ってた 方 がフォロワーさんで 渡 しにきてくれた 本 当 にありがとう 堂 々と 横 で 仕 事 しててよかった t co pvwxertu0p
2,rt legadodekonoha eis o verdadeiro significado de sonho de consumo para um fã de naruto t co w5boqyhfpo
3,para uma criança pequenina que verá um pouco mais do mundo a partir de hoje e para outra não tão pequenina que já aprendeu um pouquinho a ver o mundo com os próprios olhos t co x5k7a1dc65
4,why lesbian couples are more likely to divorce than gay ones t co ccsxngoeyl


In [ ]:
group_val = val_lda_input.groupby('Language').first().reset_index()

In [ ]:
language_code_name_pairs = pd.DataFrame()

In [ ]:
language_code_name_pairs['code', 'name'] = group.apply(lambda x : language_detect(x.Language, x.user_text), axis = 1)

In [ ]:
language_code_name_pairs